Code to accompany Machine Learning Recipes #8. We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [1]:
# importing libraries
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math

In [2]:
#importing the dataset
iris = datasets.load_iris()

#  converting dataset to pandas DataFrame
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])

#  dataset splitting into train and test
train,test = train_test_split(df,random_state = 20)

In [3]:
def class_counts(rows):
#     Counts the number of each type of example in a dataset(counts number of different classes).
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
#######
class_counts(train.values),class_counts(test.values)

({0.0: 38, 1.0: 36, 2.0: 38}, {0.0: 12, 1.0: 14, 2.0: 12})

In [4]:
def is_numeric(value):
# #     Test if a value is numeric.
    return isinstance(value, int) or isinstance(value, float)
# is_numeric(7),is_numeric("Red")

In [5]:
class Question:
#     A Question is used to partition a dataset.

#     The 'match' method is used to compare
#     the feature value in an example to the feature value stored in the
#     question.
    

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            iris.feature_names[self.column], condition, str(self.value))

In [6]:
#######
# example question
q = Question(0, 5.9)
example = train.iloc[0,:4]
q.match(example)
# iris.target[2]

True

In [7]:
def partition(rows, question):
#     Partitions a dataset.

#     For each row in the dataset, check if it matches the question. If
#     so, add it to 'true rows', otherwise, add it to 'false rows'.
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
true_rows, false_rows = partition(train.values, Question(0, 5))
# true_rows,false_rows

In [8]:
def entropy(dictionary):
    total = sum([dictionary[key] for key in dictionary])
    entropy = 0
    if total == 0:
        return 0
    else:
        for key in dictionary:
            x = dictionary[key]/total
            if x!=0:
                entropy -= x * math.log2(x)
        
    return entropy
entropy({0:1,1:1})

1.0

In [9]:
def gini(rows):
#    Calculate the Gini Impurity for a list of rows.

    counts = class_counts(rows)
    impurity = 1
    
    for key in counts:
        prob_of_lbl = counts[key] / len(rows)
        impurity -= prob_of_lbl**2
    return impurity
gini(train.values),gini(test.values)

(0.666454081632653, 0.664819944598338)

In [10]:
def gain_ratio():
    return x

In [11]:
def info_gain(left, right, current_uncertainty):
#     Calculate the Information Gain.

    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

current_uncertainty = gini(train.values)
current_uncertainty

0.666454081632653

In [12]:
true_rows, false_rows = partition(train.values, Question(0, 5))
# true_rows,false_rows
info_gain(true_rows, false_rows, current_uncertainty)

0.09356778425655965

In [13]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
#     n_features = len(rows[0]) - 1  # number of columns
    n_features = 4
    for col in range(n_features):  # for each feature

        values = list(set([item[0] for item in train.values[:,:4]]))  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
#             print(gain,question)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [14]:
#######
# Find the best question to ask first for our train dataset.
best_gain, best_question = find_best_split(train.values[:,:])
best_gain,best_question
#######

(0.29162414965986383, Is petal length (cm) >= 4.8?)

In [15]:
class Leaf:


    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [16]:
class Decision_Node:
#     A Decision Node asks a question.

#     This holds a reference to the question, and to the two child nodes.

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [17]:
def build_tree(rows):
#     Builds the tree.


    gain, question = find_best_split(rows)

# if leaf node is reached,return
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point
    
    return Decision_Node(question, true_branch, false_branch)

In [18]:
def print_tree(node, spacing=""):
    count = 0
    # Base case
    if isinstance(node, Leaf):
#         print()
        print (spacing + "Predict", node.predictions)
        count += 1
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
#     print()
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
#     print()
    print_tree(node.false_branch, spacing + "  ")

In [19]:
result_list = []
# helper function which fills the list result_list 
def list_appender(node):
#     result_list = []
    if isinstance(node, Leaf):
        result_list.append(node.predictions)
        return

    # appends the question
    result_list.append(node.question)

#     recursive calls
    list_appender(node.true_branch)

    list_appender(node.false_branch)


In [20]:
# creating a node mytree
my_tree = build_tree(train.values)

list_appender(my_tree)


In [21]:
result_list

[Is petal length (cm) >= 4.8?,
 Is petal length (cm) >= 5.2?,
 {2.0: 27},
 Is sepal length (cm) >= 6.6?,
 Is petal length (cm) >= 5.1?,
 {2.0: 1},
 {1.0: 2},
 Is sepal length (cm) >= 6.1?,
 {2.0: 5},
 Is sepal length (cm) >= 5.9?,
 Is sepal length (cm) >= 6.0?,
 Is petal length (cm) >= 5.1?,
 {1.0: 1},
 {2.0: 1},
 {1.0: 1},
 {2.0: 4},
 Is sepal length (cm) >= 5.5?,
 Is sepal width (cm) >= 4.4?,
 {0.0: 1},
 Is sepal length (cm) >= 5.6?,
 Is sepal length (cm) >= 5.9?,
 {1.0: 15},
 Is sepal length (cm) >= 5.7?,
 Is petal length (cm) >= 4.4?,
 {1.0: 1},
 Is sepal length (cm) >= 5.8?,
 {0.0: 1, 1.0: 2},
 {0.0: 1, 1.0: 3},
 {1.0: 4},
 Is petal length (cm) >= 4.4?,
 {1.0: 1},
 {0.0: 2, 1.0: 2},
 Is petal length (cm) >= 4.4?,
 {1.0: 1},
 Is sepal length (cm) >= 5.0?,
 Is sepal length (cm) >= 5.3?,
 {0.0: 5},
 Is sepal length (cm) >= 5.2?,
 {0.0: 2, 1.0: 1},
 Is sepal length (cm) >= 5.1?,
 {0.0: 5},
 {0.0: 7, 1.0: 2},
 {0.0: 14}]

In [22]:
# this cell prints the expected output from the result_list
my_counts = {0.0:0,1.0:0,2.0:0}
count = 0
for element in result_list:
    print('level: ',count)
    if isinstance(element,Question):
#         the element in the list is a question 
        for key1 in my_counts:
            print('Count of ',key1,' is: ',my_counts[key1])
        count += 1
        print('Current Entropy is: ',entropy(my_counts))
        true_rows, false_rows = partition(train.values, element)
        print('Splitting on Feature: ','"',' '.join(list(str(element).split())[1:4]),'"' ,'with gain ratio: ',info_gain(true_rows, false_rows, gini(train.values)))
        
    else:
#         the element in the list is a dictionary(node is reached)
        for key in element:
            my_counts[key] +=  element[key]
        for key in my_counts:
            print('Count of ',key,' is: ',my_counts[key])
        print('Current Entropy is ',entropy(element))
        count+=1
        print('Leaf Node Reached')
    print()

level:  0
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  0
Current Entropy is:  0
Splitting on Feature:  " petal length (cm) " with gain ratio:  0.29162414965986383

level:  1
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  0
Current Entropy is:  0
Splitting on Feature:  " petal length (cm) " with gain ratio:  0.20805072028811522

level:  2
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  27
Current Entropy is  0.0
Leaf Node Reached

level:  3
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  27
Current Entropy is:  0.0
Splitting on Feature:  " sepal length (cm) " with gain ratio:  0.08333592963321124

level:  4
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  27
Current Entropy is:  0.0
Splitting on Feature:  " petal length (cm) " with gain ratio:  0.23821747448979586

level:  5
Count of  0.0  is:  0
Count of  1.0  is:  0
Count of  2.0  is:  28
Current Entropy is  0.0
Leaf Node Reached

level:  6
Count of  0.

In [23]:
# this function returns a dictionary which has different classes and their respective confidence
def classify(row, node):

    # Base case
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
# It predicts that first row is class 1 with a confidence of 15
classify(train.values[0,:5],my_tree)

{1.0: 15}

In [24]:
# This function takes a dictionary as input and returns the key which has maximum value, i.e.
# This function returns a class which has maximum probability of being correct
def best_pred(dict):
    confidence = -9999
    dict.keys()
    res = list(dict.keys())[0]
    for key in dict:
        if dict[key] >= confidence:
            confidence = dict[key]
            res = key
    return res
# check the demo
best_pred( { 0 : 20, 1 : 21} )
# class 1 is predicted as it has higher confidence

1

In [25]:
#######
# this is a list which contains sublists of actual values and predicted values by my model
act_pred = []
for i in range(len(test)):
    act_pred.append([test.values[i,4],best_pred(classify(test.values[i,:5], my_tree))])
act_pred[:5]
#######

[[0.0, 0.0], [1.0, 1.0], [1.0, 1.0], [2.0, 2.0], [1.0, 1.0]]

In [26]:
# this function checks the number of wrong predictions from the act_pred list generated above
def accuracy(mylist):
    count = 0
    for sublist in mylist:
        count += abs(sublist[1] - sublist[0])
    print('total wrong predictions: ',count)
    return count/len(mylist)
accuracy(act_pred)

total wrong predictions:  5.0


0.13157894736842105

In [27]:
# printing actual tree
print_tree(my_tree)

Is petal length (cm) >= 4.8?
--> True:
  Is petal length (cm) >= 5.2?
  --> True:
    Predict {2.0: 27}
  --> False:
    Is sepal length (cm) >= 6.6?
    --> True:
      Is petal length (cm) >= 5.1?
      --> True:
        Predict {2.0: 1}
      --> False:
        Predict {1.0: 2}
    --> False:
      Is sepal length (cm) >= 6.1?
      --> True:
        Predict {2.0: 5}
      --> False:
        Is sepal length (cm) >= 5.9?
        --> True:
          Is sepal length (cm) >= 6.0?
          --> True:
            Is petal length (cm) >= 5.1?
            --> True:
              Predict {1.0: 1}
            --> False:
              Predict {2.0: 1}
          --> False:
            Predict {1.0: 1}
        --> False:
          Predict {2.0: 4}
--> False:
  Is sepal length (cm) >= 5.5?
  --> True:
    Is sepal width (cm) >= 4.4?
    --> True:
      Predict {0.0: 1}
    --> False:
      Is sepal length (cm) >= 5.6?
      --> True:
        Is sepal length (cm) >= 5.9?
        --> True:
        

In [28]:
# Using Sklearn DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(train.values[:,:4],train.values[:,4])
pred = model.predict(test.values[:,:4])
cm = confusion_matrix(test.values[:,4],pred)
cm
# sklearn classifier model made 3 predictions wrong and my model made 5 predictions wrong

array([[12,  0,  0],
       [ 0, 14,  0],
       [ 0,  3,  9]])